In [1]:
!pip install transformers[sentencepiece]

## BART zero-shot Text Classification

In [2]:
from transformers import pipeline 
import pandas as pd 
classifier = pipeline("zero-shot-classification", 
                      model="facebook/bart-large-mnli") 
sequence_to_classify = "one day I will see the world" 
candidate_labels = ['travel', 
                    'cooking', 
                    'dancing', 
                    'exploration'] 
result = classifier(sequence_to_classify, candidate_labels) 
pd.DataFrame(result)

,sequence,labels,scores
0,one day I will see the world,travel,0.795756
1,one day I will see the world,exploration,0.199332
2,one day I will see the world,dancing,0.002621
3,one day I will see the world,cooking,0.002291


In [3]:
result = classifier(sequence_to_classify,  
                      candidate_labels,  
                      multi_label=True) 
pd.DataFrame(result) 

,sequence,labels,scores
0,one day I will see the world,travel,0.994511
1,one day I will see the world,exploration,0.938389
2,one day I will see the world,dancing,0.005706
3,one day I will see the world,cooking,0.001819


## BART no pipeline

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer 

nli_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli") 
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli") 

In [5]:
premise = "one day I will see the world" 
label = "travel" 
hypothesis = f'This example is {label}.' 

In [6]:
x = tokenizer.encode( 
    premise, 
    hypothesis, 
    return_tensors='pt', 
    truncation_strategy='only_first') 

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


In [7]:
logits = nli_model(x)[0] 
entail_contradiction_logits = logits[:,[0,2]] 
probs = entail_contradiction_logits.softmax(dim=1) 
prob_label_is_true = probs[:,1] 
print(prob_label_is_true) 

tensor([0.9945], grad_fn=<SelectBackward>)
